## Assignment 1 
### Anubhav a1812913

#### All Imports

In [35]:
#import Libraries
import string
nltk.download('stopwords')  
from nltk.corpus import stopwords #Importing Stop Words
#Tokenize the words
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize #to TOkenize the words
# import lemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from google.colab import drive
drive.mount('/content/drive')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. Reading datasets

In [48]:
#Reading data
train_text = []
with open('drive/My Drive/NLP/Ass1/Data/reviews_train.txt', encoding='latin-1') as f:
    train_text = f.readlines()
# train_text

#reading data for testing 
test_text = []
with open('drive/My Drive/NLP/Ass1/Data/reviews_test.txt', encoding='latin-1') as f:
    test_text = f.readlines()
# test_text


#to Tokenize the words
token_words_train = [word_tokenize(text) for text in train_text]
token_words_test = [word_tokenize(text) for text in test_text]
token_words_train
token_words_test


#Seperating words into labels and training data
train_data = [i[1:-1] for i in token_words_train]
temp_train_labels = [i[0] for i in token_words_train]
train_labels = [word.replace('__label__1', 'negative').replace('__label__2', 'positive') for word in temp_train_labels]
test_data = [i[1:-1] for i in token_words_test]
temp_test_labels = [i[0] for i in token_words_test]
test_labels = [word.replace('__label__1', 'negative').replace('__label__2', 'positive') for word in temp_test_labels]


### 2. Pre-processing

In [36]:
#Remove Stop words
stop_words = set(stopwords.words('english'))
rm_stp_train  = [[word for word in line if word.lower() not in stop_words]for line in train_data]
rm_stp_test = [[word for word in line if word.lower() not in stop_words]for line in test_data]

#Remove punctuation and all non-alphanumaric characters
# rm_punct_train = [[i.translate(i.maketrans('', '', string.punctuation)) for i in line ] for line in rm_stp_train]
# rm_punct_test = [[i.translate(i.maketrans('', '', string.punctuation)) for i in line] for line in rm_stp_test]
rm_punct_train = [[word for word in line if word.isalnum()] for line in rm_stp_train]
rm_punct_test = [[word for word in line if word.isalnum()] for line in rm_stp_test]

#Creating Version 1
train_version1 = rm_punct_train
test_version1 = rm_punct_test

#Performing lowercasing of words on version1 for Version 2
train_version2 = [[word.lower() for word in line]for line in train_version1]
test_version2 = [[word.lower() for word in line]for line in test_version1]

#Performing Lemmatization of words on version2 for Version 3

train_version3 = [[lemmatizer.lemmatize(word) for word in line]for line in train_version2]
test_version3 = [[lemmatizer.lemmatize(word) for word in line]for line in test_version2]

In [37]:
#Printing Shapes of both Versions
print(f"Shape of Version 1 Training : {len(train_version1)}\n")
print(f"Shape of Version 1 Testing : {len(test_version1)}\n")
print(f"Shape of Version 2 Training : {len(train_version2)}\n")
print(f"Shape of Version 2 Testing : {len(test_version2)}\n")
print(f"Shape of Version 3 Training : {len(train_version3)}\n")
print(f"Shape of Version 3 Testing : {len(test_version3)}\n")

Shape of Version 1 Training : 300000

Shape of Version 1 Testing : 100000

Shape of Version 2 Training : 300000

Shape of Version 2 Testing : 100000

Shape of Version 3 Training : 300000

Shape of Version 3 Testing : 100000



### 3. Dataset analysis

Perform basic dataset analysis: proportion of positive and negative sentiments, number of 
unique words in all three versions. Present the results in charts or tables. Discuss the results 
and how this may affect classification performance and the selection of performance 
metrics. 

### 4. Sentiment classification using Machine Learning

Use at least two machine learning methods of your choice to classify the three versions 
obtained in point 2 into positive or negative polarity. Show and compare test results of this 
experiment. Choose the best dataset version and model and show the test results on this 
model. Discuss the results in terms of under/overfit. 

### 5. Sentiment classification using VADER sentiment lexicon

Use VADER sentiment lexicon/utility to classify the test set reviews into the polarity. 
Compare results with those obtained in point 4. Discuss the differences.

### 6. (optional challenge 3 points) Combine VADER with the classifier

(optional challenge up to 10/100 points) Combine VADER sentiment analysis output with the 
classification model. Classify best_data and compare with results with point 4

### 7. References